## XML file

### Import and read data

In [6]:
!pip install pyarrow
!pip install fastparquet
!pip install lxml



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/4.0 MB 3.3 MB/s eta 0:00:02
   --------------- ------------------------ 1.6/4.0 MB 4.6 MB/s eta 0:00:01
   ------------------------------- -------- 3.1/4.0 MB 5.5 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 6.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET


In [35]:
tree = ET.parse("geographic_data.xml")
root = tree.getroot()

xml_rows = []
for child in root:
    row = {}
    for elem in child:
        row[elem.tag] = elem.text
    xml_rows.append(row)

df_xml = pd.DataFrame(xml_rows)
df_xml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89999 entries, 0 to 89998
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   id                          89999 non-null  object
 1   state                       89999 non-null  object
 2   regional_unemployment_rate  89999 non-null  object
 3   regional_median_income      89999 non-null  object
 4   regional_median_rent        89999 non-null  object
 5   housing_price_index         89999 non-null  object
 6   cost_of_living_index        89999 non-null  object
 7   previous_zip_code           89999 non-null  object
dtypes: object(8)
memory usage: 5.5+ MB


### Cleaning

In [ ]:
#print(df_xml.info())
df_xml = df_xml.apply(lambda col: col.str.strip())
num_cols = ["regional_unemployment_rate", "regional_median_income", "regional_median_rent", "housing_price_index", "cost_of_living_index"]
for col in num_cols:
    df_xml[col] = (
        df_xml[col]
        .astype(str).str.strip()          
        .str.replace(",", "").str.replace("%", "") 
    )
    df_xml[col] = pd.to_numeric(df_xml[col], errors="coerce")
print(df_xml.info())
df_xml.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89999 entries, 0 to 89998
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          89999 non-null  object 
 1   state                       89999 non-null  object 
 2   regional_unemployment_rate  89999 non-null  float64
 3   regional_median_income      89999 non-null  int64  
 4   regional_median_rent        89999 non-null  float64
 5   housing_price_index         89999 non-null  float64
 6   cost_of_living_index        89999 non-null  float64
 7   previous_zip_code           89999 non-null  object 
dtypes: float64(4), int64(1), object(3)
memory usage: 5.5+ MB
None


,id,state,regional_unemployment_rate,regional_median_income,regional_median_rent,housing_price_index,cost_of_living_index,previous_zip_code
0,10000,OH,4.8,56000,1380.0,91.0,73.0,451
1,10001,PA,4.4,61000,1510.0,92.0,87.0,537
2,10002,VA,3.9,74000,1920.0,125.0,103.0,679
3,10003,CA,5.8,75000,1690.0,158.0,121.0,719
4,10004,WA,5.8,78000,1700.0,152.0,127.0,933


In [ ]:
df_xml['state'].unique() #all good

array(['OH', 'PA', 'VA', 'CA', 'WA', 'MI', 'TX', 'MD', 'NY', 'MO', 'IL',
       'FL', 'NJ', 'WI', 'NC', 'AZ', 'TN', 'IN', 'GA', 'MA'], dtype=object)

## Excel

### Reading

In [62]:
df_excel = pd.read_excel("loan_details.xlsx", dtype={'customer_id': 'str', 'loan_officer_id': 'str'} )
print(df_excel.info())
df_excel.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89999 entries, 0 to 89998
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   customer_id          89999 non-null  object 
 1   loan_type            89999 non-null  object 
 2   loan_amount          89999 non-null  object 
 3   loan_term            89999 non-null  int64  
 4   interest_rate        89999 non-null  float64
 5   loan_purpose         89999 non-null  object 
 6   loan_to_value_ratio  89999 non-null  float64
 7   origination_channel  89999 non-null  object 
 8   loan_officer_id      89999 non-null  object 
 9   marketing_campaign   89999 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 6.9+ MB
None


,customer_id,loan_type,loan_amount,loan_term,interest_rate,loan_purpose,loan_to_value_ratio,origination_channel,loan_officer_id,marketing_campaign
0,10000,Personal,"$17,700",36,12.50,Debt Consolidation,0.000,Direct Mail,1045,W
1,10001,mortgage,"$114,000",180,6.83,Refinance,0.774,Branch,1011,B


### Cleaning

In [63]:
print(df_excel['loan_type'].unique())
df_excel['loan_type'] = ( df_excel['loan_type'].str.strip().str.lower().str.replace('personal loan', 'personal')
                                                           .str.replace('creditcard', 'credit card')
                                                           .str.replace('cc', 'credit card'))
df_excel['loan_type'].unique()

['Personal' 'mortgage' 'Personal Loan' 'PERSONAL' 'CreditCard'
 'Credit Card' 'Mortgage' 'Home Loan' 'MORTGAGE' 'CC' 'credit card'
 'personal']


array(['personal', 'mortgage', 'credit card', 'home loan'], dtype=object)

In [64]:
df_excel['loan_amount'] = (
    df_excel['loan_amount']
    .str.strip()
    .str.replace('$', '', regex=False)
    .str.replace(',', '', regex=False)
    .str.replace(r'[$€£¥₹₽₩₺₴₦₱₫₲₵₡₮₢₨]', '', regex=True)
)
df_excel['loan_amount'] = pd.to_numeric(df_excel['loan_amount'], errors='coerce')
print(df_excel['loan_amount'].info())
df_excel.head(2)



<class 'pandas.core.series.Series'>
RangeIndex: 89999 entries, 0 to 89998
Series name: loan_amount
Non-Null Count  Dtype
--------------  -----
89999 non-null  int64
dtypes: int64(1)
memory usage: 703.2 KB
None


,customer_id,loan_type,loan_amount,loan_term,interest_rate,loan_purpose,loan_to_value_ratio,origination_channel,loan_officer_id,marketing_campaign
0,10000,personal,17700,36,12.50,Debt Consolidation,0.000,Direct Mail,1045,W
1,10001,mortgage,114000,180,6.83,Refinance,0.774,Branch,1011,B


In [ ]:
a = df_excel['loan_term'].to_list()
a  # all good
b = df_excel['interest_rate'] < 0
b.unique() # all good
df_excel['loan_purpose'].unique() # all good
df_excel['loan_to_value_ratio'].info() # all good
df_excel['origination_channel'].unique() ## all good
df_excel['marketing_campaign'].unique() # all good

<class 'pandas.core.series.Series'>
RangeIndex: 89999 entries, 0 to 89998
Series name: loan_to_value_ratio
Non-Null Count  Dtype  
--------------  -----  
89999 non-null  float64
dtypes: float64(1)
memory usage: 703.2 KB


In [85]:
print(df_excel.info())
df_excel.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89999 entries, 0 to 89998
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   customer_id          89999 non-null  object 
 1   loan_type            89999 non-null  object 
 2   loan_amount          89999 non-null  int64  
 3   loan_term            89999 non-null  int64  
 4   interest_rate        89999 non-null  float64
 5   loan_purpose         89999 non-null  object 
 6   loan_to_value_ratio  89999 non-null  float64
 7   origination_channel  89999 non-null  object 
 8   loan_officer_id      89999 non-null  object 
 9   marketing_campaign   89999 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 6.9+ MB
None


,customer_id,loan_type,loan_amount,loan_term,interest_rate,loan_purpose,loan_to_value_ratio,origination_channel,loan_officer_id,marketing_campaign
0,10000,personal,17700,36,12.50,Debt Consolidation,0.000,Direct Mail,1045,W
1,10001,mortgage,114000,180,6.83,Refinance,0.774,Branch,1011,B
